In [0]:
%pip install 'databricks-sdk==0.61.0' 'databricks-connect==16.4.2' 'pyarrow<20' 'databricks-sdk[notebook]' 'databricks-agents==1.4.0' 'mlflow<=3.2' 'mlflow[databricks]' 'databricks-vectorsearch==0.57' 'langchain==0.3.27' 'langchain-mcp' 'langchain_core==0.3.74' 'databricks-langchain==0.7.1' 'bs4' 'dotenv' 'psycopg2-binary==2.9.7' 'pgvector==0.2.5' 'langgraph==0.3.4'
import os
if os.environ.get("DATABRICKS_RUNTIME_VERSION"):
    dbutils.library.restartPython()

  Using cached databricks_sdk-0.61.0-py3-none-any.whl.metadata (39 kB)
  Using cached databricks_agents-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached databricks_vectorsearch-0.57-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_mcp-0.2.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
  Using cached databricks_langchain-0.7.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached psycopg2-binary-2.9.7.tar.gz (383 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pgvector-0.2.5-py2.py3-none-any.whl.metadata (9.9 kB)
  Using cached langgraph-0.3.4-py3-none-any.whl.metadata (17 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata 

In [0]:
%%writefile chain_postgres_genie.py
import functools
import os
import uuid
from typing import Any, Generator, Literal, Optional, Dict, List

import mlflow
import pydantic
from mlflow.models import ModelConfig
from databricks.sdk import WorkspaceClient
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    DatabricksFunctionClient,
    set_uc_function_client
)
client = DatabricksFunctionClient()
set_uc_function_client(client) 
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import create_react_agent
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from pydantic import BaseModel
from sqlalchemy import create_engine, text, event
from pgvector.psycopg2 import register_vector
from databricks.sdk import WorkspaceClient
from databricks_langchain import DatabricksEmbeddings
from databricks_langchain.chat_models import ChatDatabricks
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda
from mlflow.entities import SpanType, Document

# Enable MLflow Tracing for LangChain
mlflow.autolog()
mlflow.langchain.autolog()

# Load chain configuration provided at logging/deployment time.
model_config: ModelConfig = mlflow.models.ModelConfig()

# Pydantic models for input validation
class Message(pydantic.BaseModel):
    role: str
    content: str
    name: Optional[str] = None

class Filters(pydantic.BaseModel):
    user_name: str  # Required
    chat_id: Optional[str] = None

class CustomInputs(pydantic.BaseModel):
    filters: Filters
    k: Optional[int] = None  # Optional, will default to model_config value

class ChatRequest(pydantic.BaseModel):
    messages: List[Message]
    custom_inputs: Optional[CustomInputs] = None

class ChatResponse(pydantic.BaseModel):
    messages: List[Message]
    finish_reason: Optional[str] = None


def _get_required_env(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise RuntimeError(f"Missing required environment variable: {name}")
    return value


def get_postgres_connection(
    client: WorkspaceClient,
    db_instance_name: str,
    database_name: Optional[str] = "databricks_postgres",
) -> str:
    """
    Build a PostgreSQL SQLAlchemy URL (psycopg2) using Databricks Database credentials.

    Uses POSTGRES_GROUP env var as username if set; otherwise current user.
    Always enforces sslmode=require.
    """
    database = client.database.get_database_instance(db_instance_name)
    credentials = client.database.generate_database_credential(
        instance_names=[db_instance_name], request_id=str(uuid.uuid4())
    )

    postgres_group = os.getenv("POSTGRES_GROUP")
    username = (
        postgres_group if postgres_group else client.current_user.me().user_name
    )

    host = database.read_write_dns
    port = "5432"
    password = credentials.token
    db_name = database_name or "databricks_postgres"

    # SQLAlchemy URL with psycopg2 driver
    sqlalchemy_url = (
        f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db_name}?sslmode=require"
    )
    return sqlalchemy_url


# --- Databricks Auth (required for both embeddings and DB credentials) ---
_DATABRICKS_HOST = _get_required_env("DATABRICKS_HOST")
_DATABRICKS_TOKEN = _get_required_env("DATABRICKS_TOKEN")

workspace_client = WorkspaceClient(host=_DATABRICKS_HOST, token=_DATABRICKS_TOKEN)


# --- Postgres Engine (pgvector) ---
def _build_engine() -> Any:
    # Allow configuration via model_config or environment variables
    db_instance_name = (
        os.environ.get("DATABASE_INSTANCE_NAME")
        or model_config.get("database_instance_name")
    )
    if not db_instance_name:
        raise RuntimeError(
            "A Postgres database instance name is required. Set env 'DATABASE_INSTANCE_NAME' "
            "or include 'database_instance_name' in the model_config."
        )

    postgres_database_name = (
        os.environ.get("POSTGRES_DATABASE_NAME")
        or model_config.get("postgres_database_name")
        or "databricks_postgres"
    )

    database_url = get_postgres_connection(
        workspace_client, db_instance_name, postgres_database_name
    )

    engine = create_engine(database_url, pool_pre_ping=True)

    @event.listens_for(engine, "connect")
    def _register_vector(dbapi_connection, connection_record):  # noqa: ANN001
        # Map Python lists to pgvector type for psycopg2
        register_vector(dbapi_connection)

    return engine


engine = _build_engine()


# --- Embeddings ---
embeddings = DatabricksEmbeddings(
    endpoint=model_config.get("embedding_model"),
    token=_DATABRICKS_TOKEN,
)

# --- Vector similarity search over Postgres (pgvector) ---
@mlflow.trace
def pg_vector_similarity_search(
    query_text: str,
    k: int = 3,
    filters: Optional[Dict[str, Any]] = None,
) -> str:
    """
    Perform similarity search against message embeddings in Postgres (pgvector).

    Schema expectations:
    - message_embeddings(me: id, message_id, user_name, chat_id, embedding vector)
    - chat_history(ch: id, message_content, message_type, created_at, message_order)
    """
    filters = filters or {}

    # 1) Embed the query
    query_embedding = embeddings.embed_query(query_text)

    # 2) WHERE clause from filters
    where_conditions: List[str] = []
    params: Dict[str, Any] = {}

    if "user_name" in filters:
        where_conditions.append("me.user_name = :user_name")
        params["user_name"] = filters["user_name"]

    where_clause = ""
    if where_conditions:
        where_clause = "WHERE " + " AND ".join(where_conditions)

    # 3) Query using cosine distance operator (<=>) provided by pgvector
    sql = text(
        f"""
        SELECT
            ch.message_content,
            me.user_name,
            me.chat_id,
            ch.message_type,
            ch.created_at,
            ch.message_order,
            (me.embedding <=> CAST(:query_embedding AS vector)) AS distance
        FROM message_embeddings me
        JOIN chat_history ch ON me.message_id = ch.id
        {where_clause}
        ORDER BY me.embedding <=> CAST(:query_embedding AS vector)
        LIMIT :k
        """
    )

    span = mlflow.get_current_active_span()
    span.set_outputs([Document(page_content=sql)])

    with engine.connect() as conn:
        rows = conn.execute(
            sql, {"query_embedding": query_embedding, "k": k, **params}
        ).fetchall()

    passages = [f"Passage: {r.message_content}" for r in rows]
    return "\n".join(passages)
  

def create_context_aware_vector_search_tool(state, custom_k: Optional[int] = None):
    """Create a vector search tool that has access to user context from state"""
    
    def filtered_vector_search(query: str) -> str:
        # Extract user context from state
        user_context = state.get("user_context", {})
        filters = user_context.get("filters", {})
        
        # Use custom k if provided, otherwise fall back to model_config default
        k = custom_k if custom_k is not None else model_config.get('k')
        
        # Use your existing pg_vector_similarity_search with filters and custom k
        return pg_vector_similarity_search(
            query_text=query, 
            k=k, 
            filters=filters
        )
    
    return Tool(
        name="search_chat_history",
        description="Retrieve chat history from Postgres (pgvector) for the current user; use only if the immediate conversation context is insufficient. The input to this function should be the user message.",
        func=filtered_vector_search,
    )


genie_agent_description = model_config.get('genie_agent_description')
general_assistant_description = model_config.get('general_assistant_description')
code_agent_description = model_config.get('code_agent_description')

genie_agent = GenieAgent(
    genie_space_id=model_config.get('genie_space_id'),
    genie_agent_name="Genie",
    description=genie_agent_description,
    client=workspace_client,
    include_context=True,
)

# Max number of interactions between agents
MAX_ITERATIONS = 3

worker_descriptions = {
    "Genie": genie_agent_description,
    "General": general_assistant_description,
    "Coder": code_agent_description,
}

formatted_descriptions = "\n".join(
    f"- {name}: {desc}" for name, desc in worker_descriptions.items()
)

system_prompt = f"Decide between routing between the following workers or ending the conversation if an answer is provided. \n{formatted_descriptions}"
options = ["FINISH"] + list(worker_descriptions.keys())
FINISH = {"next_node": "FINISH"}

# Our foundation model answering the final prompt
model = ChatDatabricks(
    endpoint=model_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)

# Custom Static Tools
tools = []

def supervisor_agent(state):
    count = state.get("iteration_count", 0) + 1
    if count > MAX_ITERATIONS:
        return FINISH
    
    class nextNode(BaseModel):
        next_node: Literal[tuple(options)]

    preprocessor = RunnableLambda(
        lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
    )
    supervisor_chain = preprocessor | model.with_structured_output(nextNode)
    next_node = supervisor_chain.invoke(state).next_node
    
    # if routed back to the same node, exit the loop
    if state.get("next_node") == next_node:
        return FINISH
    return {
        "iteration_count": count,
        "next_node": next_node
    }

#######################################
# Define our multiagent graph structure
#######################################


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": name,
            }
        ]
    }


def final_answer(state):
    prompt = "Using only the content in the messages, respond to the previous user question using the answer given by the other assistant messages."
    preprocessor = RunnableLambda(
        lambda state: state["messages"] + [{"role": "user", "content": prompt}]
    )
    final_answer_chain = preprocessor | model
    return {"messages": [final_answer_chain.invoke(state)]}


def agent_node_with_context(state, agent, name, custom_k: Optional[int] = None):
    """Enhanced agent node that injects context-aware tools"""
    
    # Create the shared vector search tool with current state context and custom k
    vector_search_tool = create_context_aware_vector_search_tool(state, custom_k)
    
    if name == "Genie":
        # Genie already has its tools, just add vector search
        enhanced_agent = agent  # Genie agent already configured
        
    elif name == "Coder" or name == "General":
        # Add vector search tool to other agents' tools
        enhanced_tools = tools + [vector_search_tool]
        enhanced_agent = create_react_agent(model, tools=enhanced_tools)
        
    # Execute with enhanced agent
    result = enhanced_agent.invoke(state)
    return {
        "messages": [{
            "role": "assistant",
            "content": result["messages"][-1].content,
            "name": name,
        }]
    }

class AgentState(ChatAgentState):
    next_node: str
    iteration_count: int
    user_context: Optional[Dict[str, Any]] = None
    custom_k: Optional[int] = None

# Create enhanced agent nodes
def enhanced_genie_node(state):
    custom_k = state.get("custom_k")
    return agent_node_with_context(state, genie_agent, "Genie", custom_k)

def enhanced_coder_node(state):
    custom_k = state.get("custom_k")
    return agent_node_with_context(state, None, "Coder", custom_k)

def enhanced_general_node(state):
    custom_k = state.get("custom_k")
    return agent_node_with_context(state, None, "General", custom_k)

workflow = StateGraph(AgentState)
# Agent States
workflow.add_node("Genie", enhanced_genie_node)
workflow.add_node("Coder", enhanced_coder_node)
workflow.add_node("General", enhanced_general_node)
# Supervisor States
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("final_answer", final_answer)

workflow.set_entry_point("supervisor")
# We want our workers to ALWAYS "report back" to the supervisor when done
for worker in worker_descriptions.keys():
    workflow.add_edge(worker, "supervisor")

# Let the supervisor decide which next node to go
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next_node"],
    {**{k: k for k in worker_descriptions.keys()}, "FINISH": "final_answer"},
)
workflow.add_edge("final_answer", END)
multi_agent = workflow.compile()

###################################
# Pydantic-based PythonModel
###################################

class ChatAgentPythonModel(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.agent = multi_agent
        
    def predict(self, context, model_input: List[ChatRequest]) -> List[ChatResponse]:
        """
        Predict method with pydantic type hints for automatic schema generation.
        Note: MLflow expects List[InputType] -> List[OutputType] for pyfunc models.
        """
        responses = []
        
        for request in model_input:
            # Extract user context from request
            user_context = {}
            custom_k = None
            
            if request.custom_inputs:
                user_context["filters"] = request.custom_inputs.filters.model_dump()
                custom_k = request.custom_inputs.k
            
            # Convert pydantic messages to dict format for langgraph
            messages_dict = [msg.model_dump(exclude_none=True) for msg in request.messages]
            
            agent_request = {
                "messages": messages_dict,
                "user_context": user_context,
                "custom_k": custom_k
            }

            response_messages = []
            for event in self.agent.stream(agent_request, stream_mode="updates"):
                for node_data in event.values():
                    for msg in node_data.get("messages", []):
                        response_messages.append(Message(**msg))
            
            responses.append(ChatResponse(messages=response_messages))
        
        return responses

# Create the model instance and set it for MLflow
model_instance = ChatAgentPythonModel()
mlflow.models.set_model(model=model_instance)

Overwriting chain_postgres_genie.py


In [0]:
dbutils.widgets.text("embedding_model", "databricks-gte-large-en")
dbutils.widgets.text("database_instance_name", "tannerw-adtech-db")
dbutils.widgets.text("postgres_database_name", "databricks_postgres")
dbutils.widgets.text("llm_model_serving_endpoint_name", "databricks-claude-3-7-sonnet")
dbutils.widgets.text("target_catalog", "tanner_wendland")
dbutils.widgets.text("target_schema", "default")
dbutils.widgets.text("genie_space_id", "01efcca6fdc712d7be87a40ad4a2e33e")

In [0]:
embedding_model = dbutils.widgets.get("embedding_model")
database_instance_name = dbutils.widgets.get("database_instance_name")
postgres_database_name = dbutils.widgets.get("postgres_database_name")
llm_model_serving_endpoint_name = dbutils.widgets.get("llm_model_serving_endpoint_name")
target_catalog = dbutils.widgets.get("target_catalog")
target_schema = dbutils.widgets.get("target_schema")
genie_space_id = dbutils.widgets.get("genie_space_id")

In [0]:
import functools
import os
from typing import Any, Generator, Literal, Optional

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")
mlflow.autolog()
mlflow.langchain.autolog()

2025/09/02 20:00:40 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark: MLflow Spark dataset autologging is not supported on Databricks shared clusters or Databricks serverless clusters.
2025/09/02 20:00:40 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark.ml: [JVM_ATTRIBUTE_NOT_SUPPORTED] Attribute `sparkContext` is not supported in Spark Connect as it depends on the JVM. If you need to use this attribute, do not use Spark Connect when creating your session. Visit https://spark.apache.org/docs/latest/sql-getting-started.html#starting-point-sparksession for creating regular Spark Session in detail.


## Chain Config

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": llm_model_serving_endpoint_name,
    "embedding_model": embedding_model,
    "database_instance_name": database_instance_name,
    "postgres_database_name": postgres_database_name,
    "genie_space_id": genie_space_id,
    "k": 3,
    "genie_agent_description": "You are a an agent that can invoke Genie, a powerful text-to-sql database assistant. You can use this tool to answer questions related to sales pipelines.",
    "general_assistant_description": "The General Assistant agent is a helpful assistant that can answer any question.",
    "code_agent_description": "The Coder agent specializes in solving programming challenges, generating code snippets, debugging issues, and explaining complex coding concepts.",
}

## Chain PY

In [0]:
from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint, DatabricksGenieSpace

chain_file_path = os.path.join(os.getcwd(), 'chain_postgres_genie.py')
if not os.path.exists(chain_file_path):
    raise FileNotFoundError(f"Chain file not found at {chain_file_path}")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
print(f"Workspace URL: {workspace_url}")
os.environ['DATABRICKS_HOST'] = f"https://{workspace_url}"
os.environ['DATABRICKS_TOKEN'] = dbutils.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Pydantic-based input example - schema will be auto-inferred from type hints
# Note: MLflow pyfunc expects List[InputType], so wrap in a list
input_example = [
    {
        "messages": [{"role": "user", "content": "What was my chat history idea?"}],
        # No custom_inputs to keep it optional
    }
]

model_config = mlflow.models.ModelConfig(development_config=chain_config)

# Use pyfunc log_model with the file path (models-from-code approach)
with mlflow.start_run(run_name="adtech_chat_history_agent_postgres_genie_pydantic"):
    logged_chain_info = mlflow.pyfunc.log_model(
        python_model=chain_file_path,  # Path to the chain file
        model_config=chain_config,
        artifact_path="chat_agent_pydantic",
        input_example=input_example,  # Schema auto-inferred from List[ChatRequest] type hint
        # Specify resources for automatic authentication passthrough
        resources=[
            DatabricksServingEndpoint(endpoint_name=model_config.get("llm_model_serving_endpoint_name")),
            DatabricksGenieSpace(genie_space_id=model_config.get("genie_space_id"))
        ],
        pip_requirements=[
            "mlflow==3.2.0",
            "databricks-agents==1.4.0", 
            "databricks-langchain==0.7.1",
            "langchain==0.3.27",
            "pgvector==0.2.5",
            "psycopg2-binary==2.9.7",
            "pydantic==2.11.7",
            "sqlalchemy==2.0.43",
            "tornado==6.3.2",
            "langgraph==0.3.4"
        ]
    )

model_name = "chat_history_agent_postgres_genie"
MODEL_NAME_FQN = f"{target_catalog}.{target_schema}.{model_name}"
# Register to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=MODEL_NAME_FQN)

Workspace URL: dbc-daae04e5-a828.cloud.databricks.com


2025/09/02 20:06:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://dbc-daae04e5-a828.cloud.databricks.com/ml/experiments/3857563691880669/models/m-f220d15e7cba49d29437b5777d580e22?o=2811078624112887
/databricks/python/lib/python3.12/site-packages/databricks/connect/session.py:475: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


---------------------------------------------------------------------------
PermissionDenied                          Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-ef41dd14-648a-488c-8839-507026b1674b/lib/python3.12/site-packages/mlflow/models/utils.py:1960, in _load_model_code_path(code_path, model_config)
   1959     with _mock_dbutils(module.__dict__):
-> 1960         spec.loader.exec_module(module)
   1961 except ImportError as e:

File <frozen importlib._bootstrap_external>:995, in exec_module(self, module)

File <frozen importlib._bootstrap>:488, in _call_with_frames_removed(f, *args, **kwds)

File /Workspace/Users/tanner.wendland@databricks.com/.bundle/tannerw_adtech_demo/dev/files/data_pipelines/src/chain_postgres_genie.py:253
    251 code_agent_description = model_config.get('code_agent_description')
--> 253 genie_agent = GenieAgent(
    254     genie_space_id=model_config.get('genie_space_id'),
    255     genie_agent_name="Genie",
    256 

In [0]:
AGENT = mlflow.pyfunc.load_model(logged_chain_info.model_uri)

## Test Document Retreival

In [0]:
# Test with custom k parameter using pydantic structure
# Note: Wrap in list because MLflow pyfunc expects List[ChatRequest]
test_input_custom_k = [{
    "messages": [{"role": "user", "content": "What was my chat history idea?"}],
    "custom_inputs": {
        "filters": {"user_name": "tanner.wendland@databricks.com"},
        "k": 10  # Request 10 records instead of default 3
    }
}]

answer = AGENT.predict(test_input_custom_k)
print("=== Test with custom k=10 ===")
print(answer)

# Test with default k (no custom_inputs provided)
test_input_default = [{
    "messages": [{"role": "user", "content": "What was my chat history idea?"}]
    # No custom_inputs - should use defaults
}]

answer_default = AGENT.predict(test_input_default)
print("\n=== Test with no custom_inputs (should use defaults) ===")
print(answer_default)

# Test with required user_name but no k (should use default k)
test_input_required_user = [{
    "messages": [{"role": "user", "content": "What was my chat history idea?"}],
    "custom_inputs": {
        "filters": {"user_name": "tanner.wendland@databricks.com"}
        # No k specified - should use default from model_config
    }
}]

answer_required = AGENT.predict(test_input_required_user)
print("\n=== Test with required user_name but default k ===")
print(answer_required)

## Deploy

In [0]:
dbutils.widgets.text("secert_scope", "field-eng", "Secret Scope")
dbutils.widgets.text("secret_key", "app-secret", "Secret Key")
dbutils.widgets.text("permission_group", "Tanner W Adtech DB Access Role", "Permission Group")

In [0]:
secret_scope = dbutils.widgets.get("secert_scope")
secret_key = dbutils.widgets.get("secret_key")
permission_group = dbutils.widgets.get("permission_group")

secret_value = dbutils.secrets.get(scope=secret_scope, key=secret_key)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput, AiGatewayConfig, ServingEndpointAccessControlRequest, ServingEndpointPermissionLevel

workspace_client = WorkspaceClient()

version = uc_registered_model_info.version
serving_endpoint_name = model_name.replace(".", "-")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")

config = {
        "served_entities": [
            {
                "name": serving_endpoint_name,
                "entity_name": MODEL_NAME_FQN,
                "entity_version": version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True,
                "environment_vars": {
                    'DATABRICKS_HOST': workspace_url,
                    'DATABRICKS_TOKEN': secret_value
                }
            }
        ]
    }

ai_gateway_config = {
        'inference_table_config': {
            'enabled': True,
            'catalog_name': target_catalog,
            'schema_name': target_schema,
            'table_name': "chat_history_agent_postgres_genie_inference"
        }
    }

def does_endpoint_exists(endpoint_name):
    try:
        workspace_client.serving_endpoints.get(endpoint_name)
        return True
    except:
        return False
        
endpoint = None
if not does_endpoint_exists(serving_endpoint_name):
    print(f"Creating endpoint {serving_endpoint_name}...")
    endpoint = workspace_client.serving_endpoints.create_and_wait(
        serving_endpoint_name,
        config=EndpointCoreConfigInput.from_dict(config),
        ai_gateway=AiGatewayConfig.from_dict(ai_gateway_config)
    )
else:
    print(f"Updating endpoint {serving_endpoint_name}...")
    endpoint = workspace_client.serving_endpoints.update_config_and_wait(
        serving_endpoint_name,
        served_entities=[ServedEntityInput.from_dict(entity) for entity in config['served_entities']]
    )

# Grant permissions to the specified group after endpoint deployment
print(f"Granting permissions to group: {permission_group}")
try:
    workspace_client.serving_endpoints.set_permissions(
        serving_endpoint_id=endpoint.id,
        access_control_list=[
            ServingEndpointAccessControlRequest(group_name=permission_group, permission_level=ServingEndpointPermissionLevel.CAN_QUERY)
        ]
    )
    print(f"Successfully granted CAN_QUERY permission to group: {permission_group}")
except Exception as e:
    print(f"Failed to grant permissions to group {permission_group}: {str(e)}")
    print("Please manually grant permissions to the endpoint if needed.")